# decentralized
Primary development and execution script for evaluation of game theoretic control algorithms.

References:
1. [Fridovich-Keil - ilqgames](https://github.com/HJReachability/ilqgames/blob/master/python/dynamical_system.py)
1. [anass - ilqr](https://github.com/anassinator/ilqr)
1. [Jackson - AL iLQR Tutorial](https://bjack205.github.io/papers/AL_iLQR_Tutorial.pdf)

In [1]:
from os import remove
from pathlib import Path
import sys
sys.path.append('/mnt/c/Users/ZachJW/Documents/Grad/Research/decentralized/decentralized/')

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

from control import iLQR, LQR
from models import DoubleInt1dDynamics, DoubleInt2dDynamics
# from models import UnicycleDynamics, BicycleDynamics
from analytical_models import UnicycleDynamics, BicycleDynamics
from dynamics import MultiDynamicalModel
from cost import ObstacleCost, ReferenceCost, CouplingCost, AgentCost, GameCost
from util import Point

In [2]:
%matplotlib ipympl
import matplotlib
matplotlib.rcParams['axes.grid'] = True
np.set_printoptions(precision=3, suppress=True)

In [3]:
f1 = plt.figure(figsize=(6.76, 3.89))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
f2 = plt.figure(figsize=(7.15, 2.21))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
f3 = plt.figure()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Multi-Agent

In [6]:
dt = 0.05
N = 80
n_lqr_iter = 50
tol = 1e-3

x0 = np.array([
    -5, 0, 0, 0,
    -1, -5, 0, 0])

OBS_WEIGHT = 1.0
REF_WEIGHT = 1
COUPLING_WEIGHT = 1e6
COUPLING_RADIUS = 1.0

obstacles = []

uni1 = UnicycleDynamics(dt)
uni1_ref = ReferenceCost(xf=np.array([5, 0, 0, 0]),
                        Q=np.diag([1, 1, 0, 0]),
                        R=np.eye(2),
                        Qf=1e3*np.eye(4),
                        weight=REF_WEIGHT)
uni1_cost = AgentCost([uni1_ref] + obstacles)

uni2 = UnicycleDynamics(dt)
uni2_ref = ReferenceCost(xf=np.array([1, 5, 0, 0]),
                        Q=np.diag([1, 1, 0, 0]),
                        R=np.eye(2),
                        Qf=1e3*np.eye(4),
                        weight=REF_WEIGHT)
uni2_cost = AgentCost([uni2_ref] + obstacles)

# bike = BicycleDynamics(dt)
# bike_ref = ReferenceCost(xf=np.array([0, 5, 0, 0]),
#                          Q=np.diag([1, 1, 0, 0, 0]),
#                          R=np.eye(2),
#                          Qf=1e3*np.eye(5),
#                          weight=REF_WEIGHT)
# bike_cost = AgentCost([bike_ref] + obstacles)

dynamics = MultiDynamicalModel([uni1, uni2])

pos_inds = [(0, 1), (4, 5)]
coupling_costs = CouplingCost(pos_inds, radius=COUPLING_RADIUS, weight=COUPLING_WEIGHT)

game_cost = GameCost([uni1_cost, uni2_cost], [coupling_costs], dynamics.x_dims, dynamics.u_dims)

In [7]:
ilqr = iLQR(dynamics, game_cost, N=N)
X, U, Jf = ilqr.run(x0, n_lqr_iter, tol)

0/50	J: 220320
1/50	J: 111058	μ: 0.5	Δ: 0.5
2/50	J: 65488.4	μ: 0.125	Δ: 0.25
3/50	J: 15245.6	μ: 0.015625	Δ: 0.125
4/50	J: 14826.3	μ: 0.000976562	Δ: 0.0625
5/50	J: 13295	μ: 3.05176e-05	Δ: 0.03125
6/50	J: 7662.42	μ: 0	Δ: 0.015625
7/50	J: 7202.68	μ: 0	Δ: 0.0078125
8/50	J: 6539.16	μ: 0	Δ: 0.00390625


In [15]:
plt.figure(f1, clear=True)
ilqr.plot(X,
          Jf=Jf,
          do_headings=False, 
          surface_plot=False, 
          log_colors=True,
          coupling_radius=COUPLING_RADIUS,
          axis=(-10, 10, -10, 10),
          agent_ind=0
         )

In [16]:
plt.ylim([-5, 5])

(-5.0, 5.0)

In [17]:
def animate(f, axis=None):
    """Callback to render the current frame as an image."""
    
    if axis is None:
        axis = (-7, 7, -7, 7)
    
    ilqr.plot(X[:f],
              Jf=Jf,
              do_headings=False, 
              surface_plot=False, 
              log_colors=True,
              coupling_radius=COUPLING_RADIUS,
              axis=axis
             )
    plt.ylim(axis[-2:])

In [18]:
gifname = 'traj.gif'
plt.figure(f3, clear=True)
anim = FuncAnimation(f3, animate, frames=range(1,N+1))
anim.save(gifname, fps=5)

MovieWriter ffmpeg unavailable; using Pillow instead.


In [59]:
dist = np.linalg.norm(X[:, pos_inds[0]] - X[:, pos_inds[1]], axis=1)
crash_mask = dist < COUPLING_RADIUS

costs = np.zeros_like(dist)
for i, (x, u) in enumerate(zip(X, U)):
    costs[i] = game_cost(x, u)

In [60]:
plt.figure(f2, clear=True)
plt.clf()

f2.add_subplot(2,1,1)
plt.plot(dist)
plt.axhline(COUPLING_RADIUS, c='r')
plt.ylabel('Distance')

f2.add_subplot(2,1,2)
plt.plot(costs)
plt.ylabel('Cost')
plt.show()

### Single Agent

In [24]:
dt = 0.1
N = 60
n_lqr_iter = 50
tol = 1e-3

REF_WEIGHT = 1.0
OBS_WEIGHT = 1e6

# dynamics = DoubleInt1dDynamics(dt)
# x0 = np.array([2, 0])
# xf = np.array([0, 1])
# Q = np.eye(2)
# R = np.eye(1)

# dynamics = DoubleInt2dDynamics(dt)
# x0 = np.array([10, 10, 0, 0])
# xf = np.array([0, 0, 0, 0])
# Q = np.diag([1, 1, 0, 0])
# R = np.eye(2)

# dynamics = CarDynamics(dt)
# x0 = np.array([11, 11, np.pi/2])
# xf = np.array([0, 0, 0])
# Q = np.diag([1, 1, 0])
# R = np.eye(2)

dynamics = UnicycleDynamics(dt)
x0 = np.array([-5, -5, 0, 0])
xf = np.array([5, 5, 0, np.pi/4])
Q = np.diag([1, 1, 0, 0])
R = np.eye(2)

# dynamics = BicycleDynamics(dt)
# x0 = np.array([10, 10, np.pi/2, 0, 0])
# xf = np.array([0, 0, 0, 0, 0])
# Q = np.diag([1, 1, 0, 0, 0])
# R = np.eye(2)

Qf = np.eye(Q.shape[0]) * 1e3
reference = ReferenceCost(xf, Q, R, Qf, REF_WEIGHT)
# obstacles = [ObstacleCost((0, 1), Point(3, 3), 2, OBS_WEIGHT),
#              ObstacleCost((0, 1), Point(9, 7), 2, OBS_WEIGHT),
#              ObstacleCost((0, 1), Point(3, 10), 3, OBS_WEIGHT)]
obstacles = [ObstacleCost((0, 1), Point(0, 0), 2, OBS_WEIGHT)]

agent_cost = AgentCost(obstacles + [reference])
game_cost = GameCost([agent_cost], [], [dynamics.n_x], [dynamics.n_u])

In [25]:
ilqr = iLQR(dynamics, game_cost, N=N)
X, U, Jf = ilqr.run(x0, n_lqr_iter, tol)

0/50	J: 212617
1/50	J: 69388.1	μ: 0.5	Δ: 0.5
2/50	J: 5142.88	μ: 0.125	Δ: 0.25
3/50	J: 3948.24	μ: 0.015625	Δ: 0.125
4/50	J: 3478.34	μ: 0.000976562	Δ: 0.0625
5/50	J: 3360.17	μ: 3.05176e-05	Δ: 0.03125
6/50	J: 3268.89	μ: 0	Δ: 0.015625
7/50	J: 3246.31	μ: 0	Δ: 0.0078125
8/50	J: 3226.6	μ: 0	Δ: 0.00390625
9/50	J: 3221.47	μ: 0	Δ: 0.00195312


In [26]:
plt.figure(f1, clear=True)
ilqr.plot(X, Jf, False, 
          surface_plot=False, 
          log_colors=True,
          axis=(-6,6,-6,6)
         )

### Crazy Fly Simulation
- Position
- Orientation

### Decentralized RHC Notes
- collision radius --> incurs a higher cost
- planning radius --> defines each of the sub-problems
- for identical planning radii, we have symmetric sub-problems

#### Questions
- **When does the decentralized solution agree with the centralized one and when does it break down?**
- What is a neighbor? **Make this configurable**
    + If it's a hardcoded distance, do we need to account for relative velocity?
    + k-nearest neighbors also an option
- What is a distance between equilibria?
    + Difference between values of the potential functions
- Extensions
    + How would partially observable neighboring states influence the equilibria?
         --> stochasticity & _belief space planning_ 
    + Can we assume everyone is acting optimally?
    + Could keep track of neighboring states to minimize communication overhead

#### Ideas
- Key metric: convergence time per-node
- Optimality gap between centralized/decentralized

#### Advantages
- Realism in being able to figure things out on their own
- Scalability since matrix sizes go down in smaller state spaces
- O(N^3, A^3) for conventional dynamical games whereas O(N^3, 1) for potential game for M < N

#### Architectures

- potential impl?
      potential game
        agents
          dynamical model
          reference cost
          obstacle costs
          coupling costs

- current impl
      potential game
        multi-dynamical model
          submodels
        multi-costs
          agent-costs
            reference costs
            obstacle costs
            coupling cost

- other idea
      potential game
        dynamical models
          reference costs
        coupling costs
        obstacle costs

#### Cartesian Product State Space

**Dynamics**
```
f(x, u) =
  |  A0 A0 A0             x0     B0            |
  |  A0 A0 A0             x0     B0            |
  |  A0 A0 A0             x0     B0        u0  |
  |          A1 A1 A1 A1  x1  +     B1 B1  u1  |
  |          A1 A1 A1 A1  x1        B1 B1  u1  |
  |          A1 A1 A1 A1  x1        B1 B1      |
  |          A1 A1 A1 A1  x1        B1 B1      |
```

**Cost**
```
J(x, u) = 
    |  Lx0 Lx0 Lx0 Lx1 Lx1 Lx1 Lx1 | Lu0 Lu1 Lu1  |

H(x, u) = 
  |  Lxx0 Lxx0 Lxx0                      | Lxu0            |
  |  Lxx0 Lxx0 Lxx0                      | Lxu0            |
  |  Lxx0 Lxx0 Lxx0                      | Lxu0            |
  |                  Lxx1 Lxx1 Lxx1 Lxx1 |      Lxu1 Lxu1  |
  |                  Lxx1 Lxx1 Lxx1 Lxx1 |      Lxu1 Lxu1  |
  |                  Lxx1 Lxx1 Lxx1 Lxx1 |      Lxu1 Lxu1  |
  |                  Lxx1 Lxx1 Lxx1 Lxx1 |      Lxu1 Lxu1  |
  |  ----------------------------------------------------  |
  |  Lux0 Lux0 Lux0                      | Luu0            |
  |                  Lux1 Lux1 Lux1 Lux1 |      Luu1 Luu1  |
  |                  Lux1 Lux1 Lux1 Lux1 |      Luu1 Luu1  |
```                    